# IRF - Uppsala Python Workshop: Snakes in Space 🐍
author: Louis Richard
e-mail: louisr@irfu.se
date: 29/02/2024

## Python for data analysis:
- Numpy (https://numpy.org/doc/stable/index.html, https://numpy.org/doc/stable/user/numpy-for-matlab-users.html)
- Matplotlib (https://matplotlib.org/stable/tutorials/pyplot.html)
- Scipy (https://docs.scipy.org/doc/scipy/index.html)

## Numpy

### Create an array

In [2]:
import numpy as np

In [3]:
my_array = np.random.random((3, 3))
my_array

array([[0.06135388, 0.64595479, 0.12943096],
       [0.33164595, 0.74696515, 0.06107297],
       [0.52804931, 0.80911579, 0.8210062 ]])

In [4]:
my_array.astype(np.float32)

array([[0.06135388, 0.6459548 , 0.12943096],
       [0.33164594, 0.74696517, 0.06107297],
       [0.5280493 , 0.80911577, 0.8210062 ]], dtype=float32)

In [5]:
my_array.astype(np.float64)

array([[0.06135388, 0.64595479, 0.12943096],
       [0.33164595, 0.74696515, 0.06107297],
       [0.52804931, 0.80911579, 0.8210062 ]])

### Array slicing

In [6]:
new_array = my_array
new_array = new_array[::-1, :]
new_array[1, 1] = 50
print(new_array, my_array)

[[ 0.52804931  0.80911579  0.8210062 ]
 [ 0.33164595 50.          0.06107297]
 [ 0.06135388  0.64595479  0.12943096]] [[ 0.06135388  0.64595479  0.12943096]
 [ 0.33164595 50.          0.06107297]
 [ 0.52804931  0.80911579  0.8210062 ]]


In [7]:
copy_array = my_array.copy()
copy_array = copy_array[::-1, :]
copy_array[1, 1] = 10
print(copy_array, my_array)


[[ 0.52804931  0.80911579  0.8210062 ]
 [ 0.33164595 10.          0.06107297]
 [ 0.06135388  0.64595479  0.12943096]] [[ 0.06135388  0.64595479  0.12943096]
 [ 0.33164595 50.          0.06107297]
 [ 0.52804931  0.80911579  0.8210062 ]]


### Some functions

In [9]:
theta = np.deg2rad(30)
np.multiply(np.array([[np.cos(theta), np.sin(theta), 0], [-np.sin(theta), np.cos(theta), 0], [0, 0, 1]]), np.array([1, 0, 0]))

array([[ 0.8660254,  0.       ,  0.       ],
       [-0.5      ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ]])

In [10]:
np.matmul(np.array([[np.cos(theta), np.sin(theta), 0], [-np.sin(theta), np.cos(theta), 0], [0, 0, 1]]), np.array([1, 0, 0]))

array([ 0.8660254, -0.5      ,  0.       ])

In [11]:
r_mat = np.stack([np.array([[np.cos(theta), np.sin(theta), 0], [-np.sin(theta), np.cos(theta), 0], [0, 0, 1]]) for theta in np.random.rand(100)])

In [12]:
b = [1, 0, 0] * np.ones((100, 3))

In [13]:
m = np.random.rand(100, 3, 4, 5, 6, 8)
print(m[0, :, :, :, :, :].shape)
print(m[0].shape)
print(m[0, ...].shape)
print(m[..., 0].shape)
print(m[..., 0, :, :].shape)

(3, 4, 5, 6, 8)
(3, 4, 5, 6, 8)
(3, 4, 5, 6, 8)
(100, 3, 4, 5, 6)
(100, 3, 4, 6, 8)


In [14]:
np.matmul(r_mat[0, ...], b[0, :])

array([ 0.76911028, -0.63911609,  0.        ])

In [15]:
r_mat_b = np.matmul(r_mat, b[..., None])
print(r_mat.shape, b.shape, r_mat_b.shape)

(100, 3, 3) (100, 3) (100, 3, 1)


### datetime64

In [17]:
"2017-08-07T00:00:00.000000000"

'2017-08-07T00:00:00.000000000'

In [18]:
np.datetime64("2017-08-07T00:00:00.000000000").astype("datetime64[ns]")


numpy.datetime64('2017-08-07T00:00:00.000000000')

In [19]:
np.datetime64("2017-08-07T00:00:00.000000000").astype("datetime64[Y]")

numpy.datetime64('2017')

In [20]:
np.datetime64(0, "ns")

numpy.datetime64('1970-01-01T00:00:00.000000000')

In [21]:
print(np.datetime64(0, "ns"))
print(np.datetime64("1789-07-14T00:00:00.000000000").astype(int))
print(np.datetime64("1789-07-14T00:00:00.000000000").astype(np.int64))
print(np.datetime64("1789-07-14T00:00:00.000000000").astype(np.float64))

1970-01-01T00:00:00.000000000
-5694969600000000000
-5694969600000000000
-5.6949696e+18


In [22]:
np.datetime64("2017-08-07T00:00:00.000000000").astype("datetime64[Y]").astype("datetime64[m]")

np.datetime64("2017-08-07T00:00:00.000000000").astype(int)

1502064000000000000